In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('../00.data/corpus/labeledTrainData.tsv', sep='\t', quoting=3)
df.head(3)


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


### 데이터 탐색 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [5]:
df.review.nunique()

24904

In [6]:
# 중복 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(24904, 3)

In [8]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [9]:
# <br / > 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', '')

In [12]:
# 영어 이외의 문자는 공백으로 변환 
import re

df['review'] = df.review.apply(lambda x: re.sub('[^0-9A-Za-z]', ' ', x))

In [13]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

### Train / Test data set 만들기 

In [15]:
from sklearn.model_selection import train_test_split

feature_df = df[['review']]
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size=0.3, stratify=df.sentiment, random_state=2021
)

X_train.shape, X_test.shape

((17432, 1), (7472, 1))

In [16]:
X_train.head(3)

,review
798,Beside the fact that in all it s awesomeness...
22217,In what would be his first screenplay based ...
8296,The second Care Bears movie is immensely bett...


### Feature Vectorization and machine learning 

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

 - count Vectorizer

In [22]:
cvet = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvet.fit(X_train.review)
X_train_count = cvet.transform(X_train.review)
X_test_count = cvet.transform(X_test.review)

In [23]:
lr = LogisticRegression(C=10)
lr.fit(X_train_count, y_train)
pred = lr.predict(X_test_count)
accuracy_score(y_test, pred)

C:\Users\yongmimi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8867773019271948

 - TF/IDF Vectorizer 

In [25]:
tvet = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvet.fit(X_train.review)
X_train_tvet = tvet.transform(X_train.review)
X_test_tvet = tvet.transform(X_test.review)

In [26]:
lr = LogisticRegression(C=10)
lr.fit(X_train_tvet, y_train)
pred = lr.predict(X_test_tvet)
accuracy_score(y_test, pred)

0.8953426124197003

### 학습한 모델을 저장하고 불러오기 

In [27]:
import joblib

In [28]:
# 모델 저장 
joblib.dump(tvet, 'model/imdb_tvet.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [29]:
del tvet, lr 

In [32]:
# 모델 불러오기 
new_tvet = joblib.load('model/imdb_tvet.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [33]:
new_X_test = new_tvet.transform(X_test.review)

In [34]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8953426124197003

### pipeline을 써서 학습/예측/평가 

In [39]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))])

In [40]:
pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
accuracy_score(y_test, pred)

C:\Users\yongmimi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8867773019271948

### 최적의 하이퍼 파라미터 도출

In [41]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf',LogisticRegression())])
params = ({
    'tfidf_vect__max_df' : [100, 300, 500], 
    'lr_clf__C' : [1, 5, 10]
})

In [42]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, 
                        scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
C:\Users\yongmimi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf_vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr_clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'lr_clf__C': [1, 5, 10],
                         'tfidf_vect__max_df': [100, 300, 500]},
             scoring='accuracy', verbose=1)

In [43]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8818254817987152

In [44]:
grid_pipe.best_params_, grid_pipe.best_score_

({'lr_clf__C': 10, 'tfidf_vect__max_df': 500}, 0.8767783971147761)